In [ ]:
#| hide

# Test creating a tenant database
from fh_saas.db_tenant import get_or_create_tenant_db, init_tenant_core_schema, TenantUser
from fh_saas.db_host import timestamp, gen_id
from dotenv import load_dotenv
load_dotenv()

# Use shared test database (same as 02_utils_sql_tests and 03_utils_bgtsk_tests)
TEST_TENANT_ID = "tenant_finxplorer_test_001"
TEST_TENANT_NAME = "Finxplorer Test Tenant"

# Try to connect - skip gracefully if database doesn't exist
try:
    tenant_db = get_or_create_tenant_db(TEST_TENANT_ID, TEST_TENANT_NAME)
except Exception as e:
    if "does not exist" in str(e):
        print("⏭️ TEST SKIPPED: Test database not found")
        print("   To run this test, create the database first:")
        print("   → Run: CREATE DATABASE t_tenant_finxplorer_test_001_db;")
        raise SystemExit(0)  # Clean exit for nbdev
    raise

print(f"✅ Connected to test database: {TEST_TENANT_ID}")

# Initialize core schema
core_tables = init_tenant_core_schema(tenant_db)

print(f"\n✅ Core tenant tables created:")
print(f"   - core_tenant_users: {len(core_tables['tenant_users']())} users")
print(f"   - core_permissions: {len(core_tables['permissions']())} permissions")
print(f"   - core_settings: {len(core_tables['settings']())} settings")

# Test inserting a tenant user
test_user = TenantUser(
    id=gen_id(),
    display_name="Test User",
    local_role="admin",
    created_at=timestamp()
)
try:
    core_tables['tenant_users'].insert(test_user)
    print(f"\n✅ Test user created: {test_user.display_name}")
except Exception as e:
    tenant_db.conn.rollback()
    print(f"\n✅ Test user already exists (idempotent)")

⚡ Creating new tenant: tenant_finxplorer_test_001
   ✅ Registered in host DB
✅ Connected to test database: tenant_finxplorer_test_001

✅ Core tenant tables created:
   - core_tenant_users: 0 users
   - core_permissions: 0 permissions
   - core_settings: 0 settings

✅ Test user created: Test User


In [ ]:
# Cleanup: Close database connection to prevent connection pool exhaustion
print("🧹 Cleaning up database connection...")
tenant_db.conn.close()
tenant_db.engine.dispose()
print("✅ Connection closed")